# Neural Network

Let's try to "learn" the "exclusive or" function. Unlike "and" or "or", it cannot be learned by a single neuron/perceptron, because it is not linearly seperable.

In [1]:
import numpy as np

For two inputs, xor is true when one of them is true, and false otherwise.

In [2]:
training_inputs = np.array([
    [1, 1],
    [1, 0],
    [0, 1],
    [0, 0]
])

training_labels = np.array([0, 1, 1, 0])

In [3]:
def model(inputs, labels, learning_rate, iterations):
    parameters = init_params([2, 3, 1])
    return parameters

In [4]:
def init_params(layer_dimensions):
    num_layers = layer_dimensions
    layers_needing_weights = range(1, len(num_layers))
    
    connecting_weights = [init_weights(layer_dimensions[l], layer_dimensions[l - 1]) for l in layers_needing_weights]
    biases = [np.zeros((layer_dimensions[l], 1)) for l in layers_needing_weights]
    
    parameters = {
        'weights': connecting_weights,
        'biases': biases
    }
        
    return parameters

In [5]:
def init_weights(to_dimensions, from_dimensions):
    return np.random.randn(to_dimensions, from_dimensions)

In [6]:
parameters = model(training_inputs, training_labels, 0.25, 100)
print(parameters)

{'weights': [array([[-0.90933936, -2.14752626],
       [-1.48456353, -0.94871688],
       [-0.56848645,  1.02979642]]), array([[-0.10144614, -0.34277854, -2.28218976]])], 'biases': [array([[0.],
       [0.],
       [0.]]), array([[0.]])]}
